# Fairness in Machine Learning Example

In this notebook we will explore an example of unnmitigated machine learning to observe how a model performs for different protected classes. We will be using the Fairlearn package to calculate various metrics to help us understand how our model performs across different classes.

Check out their user guide for more information on the package! https://fairlearn.org/v0.10/user_guide/fairness_in_machine_learning.html

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/UVADS/DS1001/blob/main/code/Fairness_lab.ipynb)

In [ ]:
# You will have to run this cell to install the fairlearn package we'll be using today. You should only have to do this once (unless you restart your runtime).

!pip install fairlearn

In [ ]:
## You may have to run this line if you get an error message when you run the cell under 'Load in and explore/clean up the data'
## If you do, remove the # on the line below and run this cell

# !pip install pandas==2.0.3

In [ ]:
# import packages

import pandas as pd
import numpy as np
import fairlearn.metrics
from fairlearn.metrics import MetricFrame
from fairlearn.metrics import count, true_positive_rate, false_positive_rate, selection_rate, demographic_parity_ratio

import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) ## ignore deprecation warnings

# Adult Census Data

In this model we will be using demographic variables from census data to predict whether someone makes >50k or <=50k using data from: https://archive.ics.uci.edu/dataset/2/adult. A truncated/precleaned version of this is accessible through the `fairlearn` package, so we will import it from there.

### Load in and explore/clean up the data

In [ ]:
## import and view the data

from fairlearn.datasets import fetch_adult
census_raw = fetch_adult(as_frame=True)
census = census_raw.frame #this grabs the data in a pd.dataframe format


census.head() # this prints the first 5 lines so we can see the format of the data

In [ ]:
## create lists of categorical/numerical columns

census_catcols = list(census.select_dtypes('category')) # categorical columns

census_numcols = list(set(census.columns) - set(census_catcols)) # numerical columns

In [ ]:
## get some info on the numerical data - gives us a general idea of spread and center
census.describe().T

In [ ]:
## Visualize the spread of numeric data

fig, axs = plt.subplots(3,2)
axs = axs.ravel()
for idx,ax in enumerate(axs):
    ax.hist(census[census_numcols[idx]])
    ax.set_title(census_numcols[idx])
plt.tight_layout()

In [ ]:
## info on the categorical data
# This shows us the levels in the categories for the first 2 category columns

for col in census_catcols[:2]:
    print(census[col].value_counts(), "\n")

# Most of the columns have a ton of categories, we can combine some of them to collapse the categories.
# Typically we don't want ot have more than 5ish categories in a given column

In [ ]:
## Collapsing some categories...

# combining similar working classes
census['workclass'].replace(['Without-pay', 'Never-worked',], 'No-inc', inplace=True)
census['workclass'].replace(['Local-gov', 'State-gov', 'Federal-gov'], 'Gov', inplace=True)
# print(census['workclass'].value_counts())

# making race binary White/Non-White
census['race'] = (census.race.apply(lambda x: x if x == 'White' else "Non-White")).astype('category')
# print(census['race'].value_counts())

# combining similar education classes
census['education'].replace(['11th', '10th', '9th', '12th',], 'Some-HS', inplace=True)
census['education'].replace(['7th-8th', '5th-6th', '1st-4th', 'Preschool',], 'No-HS', inplace=True)
census['education'].replace(['Assoc-voc', 'Assoc-acdm', 'Prof-school'], 'Continued Ed', inplace=True)
census['education'].replace(['Bachelors', 'Masters', 'Doctorate'], 'College_+', inplace=True)
# print(census['education'].value_counts())

# combining similar marital statuses
census['marital-status'].replace(['Married-civ-spouse', 'Married-spouse-absent', 'Married-AF-spouse'], 'Married', inplace=True)
census['marital-status'].replace(['Divorced', 'Separated', 'Widowed'], 'Was-Married', inplace=True)
# print(census['marital-status'].value_counts())

# keeping only the top 4 countries (based on number of observations), grouping all others into "Other" category
top_country = census['native-country'].value_counts()[:5]
census['native-country'] = (census['native-country'].apply(lambda x: x if x in top_country else "Other")).astype('category')
# print(census['native-country'].value_counts())

# keeping only the top 4 occupations (based on number of observations), grouping all others into "Other" category
top_occ = census['occupation'].value_counts()[:5]
census['occupation'] = (census['occupation'].apply(lambda x: x if x in top_occ else "Other")).astype('category')
# print(census['occupation'].value_counts())


##### A little more pre-processing...

In [ ]:
# Scale numbers, One hot encode categories

census[census_numcols] = MinMaxScaler().fit_transform(census[census_numcols]) #scale the numerical values so they are all on the same scale
census_onehot = pd.get_dummies(census, columns = census_catcols) # creates dummy variables to one-hot encode all categorical variables

## One hot encoding creates a column for each category in a feature and assigns it a True/False value.
## For example, the 'workclass' column will be broken up into a column for each category ('workclass_Gov', 'workclass_No-inc', etc).
## A government workclass observation would have a True value in the 'workclass_Gov' column and a False value in all the other workclass columns.
## This is a common strategy you'll see in machine learning - also with 1/0 values instead of True/False (respectively).

In [ ]:
census_onehot.drop(['class_<=50K', 'race_White', 'sex_Male'], axis=1, inplace=True) # drop binary category duplicates
census_onehot.head() # visualize what the data looks like after being scaled/one hot encoded

Finally, we need to split it into a training set (to build our model) and testing set (to see how it performs on data it was not trained on).

We also need to split our data into our target ("class_>50K" - denoted as y) and features (everything else - denoted as x)

In [ ]:
# split the data into train and test for model

#seperate into features and target ("class_>50K")
census_x = census_onehot.loc[:, census_onehot.columns != "class_>50K"]
census_y = census_onehot.loc[:, census_onehot.columns == "class_>50K"]

#train/test split (75/25)
X_train, X_test, y_train, y_test = train_test_split(census_x, census_y, test_size=0.25, random_state=9658)

# Now, let's look at our data and model and evaluate the fairness

You will answer the following questions using the code/output below.

## Questions

### 1. The metrics we will be using in this lecture are True Positive Rate, False Positive Rate, Selection Rate, Demographic Parity Ratio, and Equalized Odds Ratio.

### 2. What are the protected classes in this dataset? Are these classes equally represented in the data?

### 3. For any given protected class, what group is being favored in the model?

### 4. Based on the fairness metrics you observed, is the model fair – why/why not?


### Looking at the data distribution

Type the name of the protected class you'd like to explore in the quotes below. Be sure to use the exact name (case sensitive!) of the column from the data frame above.

In [ ]:
protectedClass = "education" # type the protected class you'd like to explore in the quotes here

print(census[protectedClass].value_counts()) #print the number of observations in each class

#visualize the difference in class representation
plt.bar(census[protectedClass].value_counts().index.values, census[protectedClass].value_counts().values)
plt.ylabel('count')
plt.xlabel(protectedClass)
plt.title(f"Proctected Class Distribution - {protectedClass}")
plt.show()

## Model buliding

It's finally time to build our model!

We'll be building a simple logistic regression model to predict if a person makes more than 50k a year.

Basically, a logistic regression works by calculating a *probability* of an observation being in a specified class for the target variable. So in this case, our model will produce a probability of a person making more than 50k. This probability is compared to a threshold value, and if the probability is above the threshold is will be categorized as a positive outcome (in this case, making more thank 50k). For more information on logistic regressions, check out this IBM page: https://www.ibm.com/topics/logistic-regression#:~:text=Logistic%20regression%20estimates%20the%20probability,given%20dataset%20of%20independent%20variables.

In [ ]:
## train model

lreg = LogisticRegression() #initialize a logistic regression model
lreg.fit(X_train, y_train) #train this model using our training data

y_pred = lreg.predict(X_test) # store predicted values for the test set

#### Average accuracy on test data

In [ ]:
print("Average accuracy on test data:\t",round(lreg.score(X_test, y_test)*100,2),"%")


### Fairness Metrics

We are using the Fairlearn package in Python.

You will need to understand what the metric used below mean and how they are calculated. You can find information on the functions used in their documentation: https://fairlearn.org/v0.10/api_reference/index.html#module-fairlearn.metrics

In [ ]:
# Construct a function dictionary with the metrics we'd like for each class
my_metrics = {
    'true positive rate' : true_positive_rate,
    'false positive rate' : false_positive_rate,
    'selection rate' : selection_rate,
    'count' : count
}
# Construct a MetricFrame for race
mf_race = MetricFrame(
    metrics=my_metrics,
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=X_test["race_Non-White"]
)

# Construct a MetricFrame for sex
mf_sex = MetricFrame(
    metrics=my_metrics,
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=X_test["sex_Female"]
)

In [ ]:
# Display the by_group breakdown for race
print("Metrics by Race:")
print(mf_race.by_group)

In [ ]:
def create_confmatrix(y_test, y_pred):
    '''
    creates a confusion matrix with more descriptive formatting
    '''
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel() # grab the individual values

    # create a data frame with the values in the correct spots
    conf_matrix = pd.DataFrame({'predicted positive': [tp, fp],
                                'predicted negative': [fn, tn]},
                                index=['actual positive','actual negative'])

    # return the dataframe to be saved/viewed
    return conf_matrix

#### Overall metrics

Here is the confusion matrix for the model overall with *counts* for the true positive, false positive, true negative, and false negative.

For more information on confusion matrices, check out the wiki page: https://en.wikipedia.org/wiki/Confusion_matrix

In [ ]:
# overall confusion matrix
print("Confusion matrix for all test data:")
create_confmatrix(y_test, y_pred)

This code works because it's displaying the overall (aggregate) fairness metrics from the `MetricFrame` object `mf_race` in a formatted DataFrame.

Here's why it works:

1. **`mf_race.overall`**: This attribute contains a dictionary-like object with the overall metrics (averaged across all groups) that were calculated when the `MetricFrame` was created. These metrics include:
    - true positive rate
    - false positive rate
    - selection rate
    - count

2. **`pd.DataFrame(..., columns = ["overall"])`**: Converts the metrics dictionary into a pandas DataFrame with a single column named "overall"

3. **`.T`**: Transposes the DataFrame so that instead of having metrics as rows, they become columns, making it easier to read and compare with the by-group breakdowns shown in later cells

The result is a clean, single-row DataFrame showing the model's performance metrics aggregated across all test data, which serves as a baseline for comparison when you look at how the model performs for different protected classes (race and sex) in the cells below.

This is particularly useful in fairness analysis because you can compare these overall metrics to the by-group metrics (like `mf_race.by_group`) to identify disparities in model performance across different demographic groups.

In [ ]:
## The overall metrics. You'll use these to compare to with the metrics broken down by each protected class below.
## Think about how the differing performance would impact that group based on your understanding of each metric.


Now we can look at fairness metrics for each protected class:

#### Race

In [ ]:
## metrics broken down by race classes. Compare these to the metrics above.
mf_race.by_group

Definitions

1. Demographic Parity Ratio: Measures whether different groups (Male vs Female) are selected at similar rates by the model. A ratio of 1.0 means perfect parity - both groups are predicted to earn >50K at the same rate. Values below 1.0 indicate one group is favored over another. For example, 0.5 means one group is selected at half the rate of the other.
2. Equality of Odds Ratio: The equalized odds ratio of 1 means that all groups have the same true positive, true negative, false positive, and false negative rates

In [ ]:
# Derived fairness metrics. Be sure you understand the scale and meaning of these.

dpr_race = fairlearn.metrics.demographic_parity_ratio(y_test, y_pred, sensitive_features=X_test.filter(regex="race.*"))
print("Demographic Parity ratio:\t", dpr_race)

eodds_race = fairlearn.metrics.equalized_odds_ratio(y_test, y_pred, sensitive_features=X_test.filter(regex="race.*"))
print("Equalized Odds ratio:\t\t", eodds_race)

#### Sex

In [ ]:
## metrics broken down by sex classes. Compare these to the metrics above.

mf_sex.by_group

In [ ]:
# Derived fairness metrics. Be sure you understand the scale and meaning of these.

dpr_sex = fairlearn.metrics.demographic_parity_ratio(y_test, y_pred, sensitive_features=X_test.filter(regex="sex.*"))
print("Demographic Parity ratio:\t", dpr_sex)

eodds_sex = fairlearn.metrics.equalized_odds_ratio(y_test, y_pred, sensitive_features=X_test.filter(regex="sex.*"))
print("Equalized Odds ratio:\t\t", eodds_sex)

In [ ]:
# Let's break down the demographic parity ratio for sex with detailed visualizations

# First, let's see the selection rates for each sex group
print("=" * 60)
print("DEMOGRAPHIC PARITY RATIO EXPLANATION")
print("=" * 60)
print("\nThe Demographic Parity Ratio compares the selection rates")
print("between different groups (Male vs Female).")
print("\nFormula: min(rate_group1, rate_group2) / max(rate_group1, rate_group2)")
print("A value of 1.0 = perfect parity (both groups selected equally)")
print("A value < 1.0 = disparity exists (one group favored over another)")
print("=" * 60)

# Calculate selection rates for each sex group
male_mask = X_test["sex_Female"] == False
female_mask = X_test["sex_Female"] == True

male_selection_rate = y_pred[male_mask].sum() / len(y_pred[male_mask])
female_selection_rate = y_pred[female_mask].sum() / len(y_pred[female_mask])

print(f"\n📊 SELECTION RATES BY SEX:")
print(f"Male (predicted >50K):   {male_selection_rate:.4f} ({male_selection_rate*100:.2f}%)")
print(f"Female (predicted >50K): {female_selection_rate:.4f} ({female_selection_rate*100:.2f}%)")
print(f"\nDemographic Parity Ratio: {dpr_sex:.4f}")
print(f"Interpretation: Females are selected at {dpr_sex*100:.2f}% the rate of males")




In [ ]:
# Visualization 1: Selection rates comparison
fig, axes = plt.subplots(1, 3, figsize=(16, 5))


# Plot 1: Selection rates bar chart
categories = ['Male', 'Female']
selection_rates = [male_selection_rate, female_selection_rate]
colors = ['#3498db', '#e74c3c']

axes[0].bar(categories, selection_rates, color=colors, alpha=0.7, edgecolor='black')
axes[0].set_ylabel('Selection Rate (Predicted >50K)', fontsize=11)
axes[0].set_title('Selection Rates by Sex\n(What % are predicted to earn >50K?)', fontsize=12, fontweight='bold')
axes[0].set_ylim(0, max(selection_rates) * 1.2)
for i, v in enumerate(selection_rates):
    axes[0].text(i, v + 0.01, f'{v*100:.1f}%', ha='center', fontweight='bold')
axes[0].axhline(y=male_selection_rate, color='gray', linestyle='--', alpha=0.5, label='Male rate (reference)')
axes[0].legend()

# Plot 2: Counts of predictions
male_positive = y_pred[male_mask].sum()
male_negative = (~y_pred[male_mask]).sum()
female_positive = y_pred[female_mask].sum()
female_negative = (~y_pred[female_mask]).sum()

x = np.arange(2)
width = 0.35

axes[1].bar(x - width/2, [male_positive, female_positive], width, label='Predicted >50K', color='#2ecc71', alpha=0.8)
axes[1].bar(x + width/2, [male_negative, female_negative], width, label='Predicted ≤50K', color='#95a5a6', alpha=0.8)
axes[1].set_ylabel('Count of Predictions', fontsize=11)
axes[1].set_title('Distribution of Predictions by Sex', fontsize=12, fontweight='bold')
axes[1].set_xticks(x)
axes[1].set_xticklabels(categories)
axes[1].legend()

# Add count labels
for i, (pos, neg) in enumerate([(male_positive, male_negative), (female_positive, female_negative)]):
    axes[1].text(i - width/2, pos + 50, str(pos), ha='center', fontweight='bold', fontsize=9)
    axes[1].text(i + width/2, neg + 50, str(neg), ha='center', fontweight='bold', fontsize=9)

# Plot 3: Visual representation of disparity
axes[2].barh(['Female', 'Male'], [female_selection_rate, male_selection_rate],
             color=['#e74c3c', '#3498db'], alpha=0.7, edgecolor='black')
axes[2].set_xlabel('Selection Rate', fontsize=11)
axes[2].set_title(f'Disparity Visualization\nDPR = {dpr_sex:.4f}', fontsize=12, fontweight='bold')
axes[2].axvline(x=male_selection_rate, color='gray', linestyle='--', alpha=0.5, label='Male rate (reference)')

# Add arrows showing the gap
gap = male_selection_rate - female_selection_rate
axes[2].annotate('', xy=(male_selection_rate, 0), xytext=(female_selection_rate, 0),
                arrowprops=dict(arrowstyle='<->', color='red', lw=2))
axes[2].text((male_selection_rate + female_selection_rate)/2, 0.05,
             f'Gap: {gap*100:.1f}%', ha='center', color='red', fontweight='bold')

plt.tight_layout()
plt.show()

# Additional statistics
print(f"\n📈 DETAILED STATISTICS:")
print(f"\nSample sizes:")
print(f"  Male:   {male_mask.sum()} observations")
print(f"  Female: {female_mask.sum()} observations")

print(f"\nPredictions breakdown:")
print(f"  Male predicted >50K:   {male_positive} ({male_selection_rate*100:.1f}%)")
print(f"  Female predicted >50K: {female_positive} ({female_selection_rate*100:.1f}%)")

print(f"\n⚠️  FAIRNESS ASSESSMENT:")
if dpr_sex >= 0.8:
    print("✓ Passes the 80% rule (commonly used fairness threshold)")
else:
    print(f"✗ Fails the 80% rule (DPR should be ≥ 0.8)")
    print(f"  Current DPR ({dpr_sex:.4f}) indicates significant disparity")
    print(f"  Females are underrepresented in positive predictions")

In [ ]:
# Equalized Odds breakdown for sex (similar style to DPR breakdown above)

print("=" * 60)
print("EQUALIZED ODDS RATIO EXPLANATION")
print("=" * 60)
print("Equalized Odds requires similar error and success rates across groups.")
print("We compare True Positive Rate (TPR), False Positive Rate (FPR),")
print("True Negative Rate (TNR), and False Negative Rate (FNR) for Male vs Female.")
print(f"Equalized Odds Ratio (overall): {eodds_sex:.4f}")
print("A value of 1.0 indicates parity across all these rates.\n")

y_true = y_test["class_>50K"]

def group_confusion(mask):
    tp = np.sum((y_pred == True)  & (y_true == True)  & mask)
    fp = np.sum((y_pred == True)  & (y_true == False) & mask)
    tn = np.sum((y_pred == False) & (y_true == False) & mask)
    fn = np.sum((y_pred == False) & (y_true == True)  & mask)
    return tp, fp, tn, fn

male_tp, male_fp, male_tn, male_fn = group_confusion(male_mask)
female_tp, female_fp, female_tn, female_fn = group_confusion(female_mask)

def rates(tp, fp, tn, fn):
    tpr = tp / (tp + fn) if (tp + fn) else np.nan
    fpr = fp / (fp + tn) if (fp + tn) else np.nan
    tnr = tn / (tn + fp) if (tn + fp) else np.nan
    fnr = fn / (fn + tp) if (fn + tp) else np.nan
    return tpr, fpr, tnr, fnr

male_tpr, male_fpr, male_tnr, male_fnr = rates(male_tp, male_fp, male_tn, male_fn)
female_tpr, female_fpr, female_tnr, female_fnr = rates(female_tp, female_fp, female_tn, female_fn)

print("Group performance rates:")
print(f"Male   -> TPR:{male_tpr:.3f} FPR:{male_fpr:.3f} TNR:{male_tnr:.3f} FNR:{male_fnr:.3f}")
print(f"Female -> TPR:{female_tpr:.3f} FPR:{female_fpr:.3f} TNR:{female_tnr:.3f} FNR:{female_fnr:.3f}\n")

# Ratios (smaller / larger for each metric)
def ratio(a, b):
    return min(a, b) / max(a, b) if (a > 0 and b > 0) else np.nan

tpr_ratio = ratio(male_tpr, female_tpr)
fpr_ratio = ratio(male_fpr, female_fpr)
tnr_ratio = ratio(male_tnr, female_tnr)
fnr_ratio = ratio(male_fnr, female_fnr)

print("Per-metric parity ratios (min/max):")
print(f"TPR ratio: {tpr_ratio:.3f}")
print(f"FPR ratio: {fpr_ratio:.3f}")
print(f"TNR ratio: {tnr_ratio:.3f}")
print(f"FNR ratio: {fnr_ratio:.3f}")

worst_ratio = np.nanmin([tpr_ratio, fpr_ratio, tnr_ratio, fnr_ratio])
print(f"\nWorst-case parity ratio (approx basis of Equalized Odds): {worst_ratio:.3f}")
print(f"Reported Equalized Odds Ratio: {eodds_sex:.4f}\n")

print("Fairness assessment:")
if eodds_sex >= 0.8:
    print("Passes a common 0.8 threshold.")
else:
    print("Fails a common 0.8 threshold; substantial disparity exists.")

# Visualization
fig2, axarr = plt.subplots(1, 3, figsize=(16,5))

# Panel 1: TPR/FPR comparison
metrics_names = ["TPR","FPR","TNR","FNR"]
male_vals = [male_tpr, male_fpr, male_tnr, male_fnr]
female_vals = [female_tpr, female_fpr, female_tnr, female_fnr]
x_pos = np.arange(len(metrics_names))
axarr[0].bar(x_pos - 0.2, male_vals, width=0.4, label="Male", color="#3498db")
axarr[0].bar(x_pos + 0.2, female_vals, width=0.4, label="Female", color="#e74c3c")
axarr[0].set_xticks(x_pos)
axarr[0].set_xticklabels(metrics_names)
axarr[0].set_ylabel("Rate")
axarr[0].set_title("Error/Success Rates by Sex")
axarr[0].legend()

# Panel 2: Confusion matrix counts per group (normalized)
def norm_counts(tp, fp, tn, fn):
    total = tp + fp + tn + fn
    return [tp/total, fp/total, tn/total, fn/total]
male_norm = norm_counts(male_tp, male_fp, male_tn, male_fn)
female_norm = norm_counts(female_tp, female_fp, female_tn, female_fn)
labels_cm = ["TP","FP","TN","FN"]
x2 = np.arange(len(labels_cm))
axarr[1].bar(x2 - 0.2, male_norm, 0.4, label="Male", color="#3498db")
axarr[1].bar(x2 + 0.2, female_norm, 0.4, label="Female", color="#e74c3c")
axarr[1].set_xticks(x2)
axarr[1].set_xticklabels(labels_cm)
axarr[1].set_ylabel("Proportion")
axarr[1].set_title("Normalized Confusion Components")
axarr[1].legend()

# Panel 3: Parity ratios
parity_vals = [tpr_ratio, fpr_ratio, tnr_ratio, fnr_ratio]
axarr[2].bar(metrics_names, parity_vals, color="#8e44ad", alpha=0.7)
axarr[2].axhline(1.0, color="gray", linestyle="--", linewidth=1)
axarr[2].axhline(0.8, color="red", linestyle="--", linewidth=1)
axarr[2].set_ylim(0, 1.05)
axarr[2].set_ylabel("Parity Ratio (min/max)")
axarr[2].set_title(f"Per-Metric Parity Ratios\nWorst={worst_ratio:.3f} | EO={eodds_sex:.3f}")

plt.tight_layout()
plt.show()